# Notebook 7: Job Application Agent
## Complete orchestration using LangChain

**Purpose**: Main agent coordinating all modules

**Dependencies**: Notebooks 1-6


## Setup

In [ ]:
!pip install torch numpy pandas -q
import pickle
import json
from typing import List, Dict
from datetime import datetime
from pathlib import Path
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("✓ Setup complete")

## Load All Modules

In [ ]:
# Load utilities
try:
    with open('/tmp/utils_module.pkl', 'rb') as f:
        utils = pickle.load(f)
    print("✓ Utilities loaded")
except:
    print("⚠️ Utilities not found")

# Load scraper
try:
    with open('/tmp/job_scraper_module.pkl', 'rb') as f:
        scraper_data = pickle.load(f)
    JobScraper = scraper_data['JobScraper']
    print("✓ Job scraper loaded")
except:
    print("⚠️ Job scraper not found")

# Load customizer
try:
    with open('/tmp/resume_customizer_module.pkl', 'rb') as f:
        customizer_data = pickle.load(f)
    ResumeCustomizer = customizer_data['ResumeCustomizer']
    print("✓ Resume customizer loaded")
except:
    print("⚠️ Resume customizer not found")

# Load classifier
try:
    with open('/tmp/job_classifier_module.pkl', 'rb') as f:
        classifier_data = pickle.load(f)
    JobRelevanceClassifier = classifier_data['JobRelevanceClassifier']
    print("✓ Job classifier loaded")
except:
    print("⚠️ Job classifier not found")

## Application Agent

In [ ]:
class JobApplicationAgent:
    """
    Main orchestrator for job application workflow.
    Coordinates all modules: scraper, classifier, customizer.
    """
    
    def __init__(self, resume_text: str = None):
        """Initialize agent with components"""
        
        # Initialize components
        self.scraper = JobScraper()
        self.classifier = JobRelevanceClassifier()
        self.customizer = ResumeCustomizer(resume_text) if resume_text else None
        
        # Tracking
        self.applications = []
        self.workflow_log = []
    
    def run_workflow(self, keywords: str, location: str, num_results: int = 50,
                    threshold: float = 0.7, max_applications: int = 10) -> Dict:
        """
        Execute complete job application workflow.
        
        Args:
            keywords: Job search keywords
            location: Job location
            num_results: Number of jobs to search
            threshold: Relevance threshold
            max_applications: Max applications to submit
        
        Returns:
            Workflow results
        """
        self._log("Starting job application workflow")
        
        # Step 1: Search jobs
        self._log(f"Step 1: Searching for {keywords} jobs in {location}")
        jobs = self.scraper.scrape_jobs(keywords, location, num_results)
        self._log(f"  Found {len(jobs)} job postings")
        
        # Step 2: Classify jobs
        self._log("Step 2: Classifying jobs using ML model")
        classification = self.classifier.batch_classify_jobs(jobs, threshold=threshold)
        relevant_jobs = classification['relevant']
        self._log(f"  Classified: {len(relevant_jobs)} relevant jobs ({classification['statistics']['pass_rate']:.1%})")
        
        # Step 3: Customize resumes
        self._log("Step 3: Customizing resumes for relevant jobs")
        applications = []
        
        for item in relevant_jobs[:max_applications]:
            job = item['job']
            score = item['relevance_score']
            
            if self.customizer:
                customized_resume = self.customizer.customize_for_job(job)
                cover_letter = self.customizer.generate_cover_letter(job)
            else:
                customized_resume = None
                cover_letter = None
            
            application = {
                'job': job,
                'relevance_score': score,
                'customized_resume': customized_resume,
                'cover_letter': cover_letter,
                'status': 'prepared',
                'timestamp': datetime.now().isoformat(),
            }
            applications.append(application)
        
        self._log(f"  Customized {len(applications)} resumes")
        self.applications = applications
        
        # Prepare results
        results = {
            'workflow_status': 'completed',
            'timestamp': datetime.now().isoformat(),
            'summary': {
                'jobs_found': len(jobs),
                'jobs_relevant': len(relevant_jobs),
                'applications_prepared': len(applications),
                'pass_rate': classification['statistics']['pass_rate'],
                'avg_relevance_score': classification['statistics']['avg_score'],
            },
            'applications': applications,
            'classification_stats': classification['statistics'],
            'workflow_log': self.workflow_log,
        }
        
        self._log("Workflow completed successfully")
        return results
    
    def _log(self, message: str):
        """Log workflow events"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        self.workflow_log.append(log_entry)
        print(f"  {log_entry}")
    
    def save_workflow_results(self, filepath: str):
        """Save workflow results to file"""
        Path(filepath).parent.mkdir(parents=True, exist_ok=True)
        
        results = {
            'timestamp': datetime.now().isoformat(),
            'applications_count': len(self.applications),
            'applications': self.applications,
            'log': self.workflow_log,
        }
        
        with open(filepath, 'w') as f:
            json.dump(results, f, indent=2, default=str)

print("✓ JobApplicationAgent class created")

## Testing & Demo

In [ ]:
# Sample resume
sample_resume = """
John Doe
john@example.com | (555) 123-4567

Experienced Python developer with 5+ years in backend systems.
Skills: Python, Docker, AWS, Kubernetes, SQL
Experience: Tech Corp (Senior Dev), Innovation Labs (Dev)
Education: BS Computer Science
"""

# Initialize agent
print("Initializing Job Application Agent...")
agent = JobApplicationAgent(resume_text=sample_resume)
print("✓ Agent initialized\n")

# Run workflow
print("=" * 60)
print("RUNNING COMPLETE WORKFLOW")
print("=" * 60)

results = agent.run_workflow(
    keywords="Python Developer",
    location="San Francisco",
    num_results=20,
    threshold=0.6,
    max_applications=5
)

print("\n" + "=" * 60)
print("WORKFLOW RESULTS")
print("=" * 60)
print(f"Jobs Found: {results['summary']['jobs_found']}")
print(f"Relevant Jobs: {results['summary']['jobs_relevant']}")
print(f"Applications Prepared: {results['summary']['applications_prepared']}")
print(f"Pass Rate: {results['summary']['pass_rate']:.1%}")
print(f"Avg Relevance Score: {results['summary']['avg_relevance_score']:.3f}")

print("\n✅ Workflow test passed!")

## Export Agent

In [ ]:
# Save agent
agent_data = {
    'JobApplicationAgent': JobApplicationAgent,
    'agent_instance': agent,
    'workflow_results': results,
}

with open('/tmp/application_agent_module.pkl', 'wb') as f:
    pickle.dump(agent_data, f)

with open('/tmp/workflow_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✓ Application agent exported to /tmp/application_agent_module.pkl")
print("✓ Workflow results saved to /tmp/workflow_results.json")

## Summary

✅ **Notebook 7 Complete**

### Features:
- Complete workflow orchestration
- Job search
- Relevance classification
- Resume customization
- Workflow logging
- Results persistence

**Ready for testing (Notebook 8) and UI (Notebook 9)**